# Day 4, Giant Squid
Bingo is played on a set of boards each consisting of a 5x5 grid of numbers. Numbers are chosen at random, and the chosen number is marked on all boards on which it appears. (Numbers may not appear on all boards.) If all numbers in any row or any column of a board are marked, that board wins. (Diagonals don't count.)

In [ ]:
let sample = "7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7"

let inline readInData (file:string) = file.Split(Environment.NewLine) |> Array.toList

let inline splitText (delim:string) (l:string) =
    l.Split(delim) |> Array.where (fun v -> v.Length > 0) |> Array.map int |> Array.toList
let splitLine = splitText Environment.NewLine
let splitComma = splitText ","
let splitSpace = splitText " "
let allBelowZero = List.tryFindIndex (fun data -> data |> List.forall (fun i -> i < 0))
let inline sumBoard b = b |> List.map (fun l -> l|> List.where (fun n -> n > 0) |> List.sum ) |> List.sum

/// Check a 5x5 board (5 arrays of 5) for a given number and negate it
let inline checkBoard num (board:List<List<int>>) =
    board |> List.map (fun row -> row |> List.map (fun col -> if col = num then col - 1000 else col))

let checkBoards num = List.map (fun b -> checkBoard num b)

/// Check that a given board has either a row with all numbers less than 0 or a column with all numbers less than 0.
/// If there is, return the winning numbers and the board
let checkForWinner (board:List<List<int>>) =
    match board |> allBelowZero with
    | Some (indx) -> Some (indx, board)
    | None -> 
        let tr = board |> List.transpose
        match tr |> allBelowZero with
        | Some (indx) -> Some (indx, tr)
        | None -> None

let rec playBingo (numbers:List<int>) (lastCall:int,boards:List<List<List<int>>>) =
    match boards |> List.tryPick checkForWinner with
    | None ->
        match numbers with
        //  keep going unless we've cleaned out our number list
        | [] -> (-1, boards)
        | head::tail -> (head,boards) ||> checkBoards |> (fun b -> playBingo tail (head,b))
    //  We have a winner
    | Some (i,x) -> (lastCall,[x |> List.removeAt i])

In [ ]:
System.IO.File.ReadAllText("../Data/Day4.txt") |> readInData 
|> (fun (h::t) ->
    let calls = splitComma h
    let boards = t |> List.where (fun l -> l.Length > 0) |> List.map splitSpace |> List.chunkBySize 5
    (calls,(0,boards)) ||> playBingo 
        |> (fun (lc, b) -> 
            let boardTotal = b |> List.head |> sumBoard
            boardTotal * lc)
    )

25023

In [ ]:
let checkForWinner2 (board:List<List<int>>) =
    match board |> allBelowZero with
    | Some (indx) -> true
    | None -> 
        let tr = board |> List.transpose
        match tr |> allBelowZero with
        | Some (indx) -> true
        | None -> false

let rec playBingo2 (numbers:List<int>) (lastCall:int,boards:List<List<List<int>>>) =
    match boards |> List.partition checkForWinner2 with
    | (winners,losers) when (List.length losers) = 0 -> (lastCall, winners)
    | (winners,losers) ->
        match numbers with
        | [] -> (lastCall, losers)
        | head::tail -> (head,losers) ||> checkBoards |> (fun b -> playBingo2 tail (head,b))

System.IO.File.ReadAllText("../Data/Day4.txt") |> readInData 
    |> (fun (h::t) ->
        let calls = splitComma h
        let boards = t |> List.where (fun l -> l.Length > 0) |> List.map splitSpace |> List.chunkBySize 5
        (calls,(-1,boards)) ||> playBingo2 
            |> (fun (lc, b) -> 
                let boardTotal = b |> List.head |> sumBoard
                boardTotal * lc)
    )

1924